## Scenario 3: Tracking server on localhost with SQLite backend and remote (s3) artifacts store

MLflow setup:
- Tracking server: yes, remote server (EC2)
- Backend store: sqlite database (db name: mlflow_db, port: 5432)
- Artifacts store: S3 bucket (mlops-03-artifacts)

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db --default-artifact-root s3://<YOUR_BUCKET>`

The exampe uses an s3 bucket to store the artifacts, so you'll to provide the bucket. Check the [mlflow documentation](https://mlflow.org/docs/latest/tracking.html#artifact-stores) for more options to store your artifacs.

Master username

mlflow

Master password

yesmXzrEISbtUNIoc3rK

DB Endpoint:
mlflow-db.cvsmgmqedvt7.us-east-2.rds.amazonaws.com

In [11]:
import mlflow
import os

os.environ["AWS_PROFILE"] = "default"

TRACKING_SERVER_HOST = "ec2-18-224-138-85.us-east-2.compute.amazonaws.com"
mlflow.set_tracking_uri(f"http://{TRACKING_SERVER_HOST}:5000")

In [6]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://ec2-18-224-138-85.us-east-2.compute.amazonaws.com:5000'


In [7]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlops-03-artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlops-03-artifacts/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")

In [13]:
mlflow.list_experiments()

[<Experiment: artifact_location='s3://mlops-03-artifacts/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='s3://mlops-03-artifacts/1', experiment_id='1', lifecycle_stage='active', name='my-experiment-1', tags={}>]

### Interacting with the model registry

In [18]:
from mlflow.tracking import MlflowClient


# client = MlflowClient("http://127.0.0.1:5000")
client = MlflowClient(f"http://{TRACKING_SERVER_HOST}:5000")

In [19]:
client.list_registered_models()

[]

In [20]:
run_id = client.list_run_infos(experiment_id='1')[0].run_id
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2022/06/06 16:58:44 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: creation_timestamp=1654549124964, current_stage='None', description='', last_updated_timestamp=1654549124964, name='iris-classifier', run_id='fe377bf4c4904bf9858e0df3314a5d02', run_link='', source='s3://mlops-03-artifacts/1/fe377bf4c4904bf9858e0df3314a5d02/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>